### Load the dataset

In [163]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()
torch.cuda.synchronize()

In [164]:
import os
import numpy as np
import torch, torchvision
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Subset, DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from typing import List
from PIL import Image
from IPython.display import display
from sklearn.model_selection import train_test_split
import time

ASSET_DIR = "../data/assets/imagen1"

In [165]:
### INSPIRED BY VGG16 architecture
def get_model(num_classes: int) -> nn.Module:
    res = nn.Sequential(
        nn.Conv2d(3, 8, kernel_size=3, padding=1),
        nn.LeakyReLU(0.1),
        nn.BatchNorm2d(8),
        nn.MaxPool2d(2),
        nn.Conv2d(8, 16, kernel_size=3, padding=1),
        nn.LeakyReLU(0.1),
        nn.BatchNorm2d(16),
        nn.MaxPool2d(2),

        nn.Flatten(),

        nn.Linear(3136, 70),
        nn.LeakyReLU(0.1),
        nn.Dropout(p=0.3),
        nn.Linear(70, num_classes)
    )
    return res



In [166]:
# Auto‑generate a loader snippet for the trained PyTorch model
from utils import generate_torch_loader_snippet

example_input = torch.tensor(
    [[0.0, 0.0]], dtype=torch.float32
)  # minimal example for tracing if needed
snippet = generate_torch_loader_snippet(
    model=get_model(21), prefer="auto"
, compression="zlib")

with open("image_model.py", "w") as f:
    f.write(snippet)

In [167]:
def get_augmentations():
    T = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(58),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    ])
    return T

In [168]:
dataset = ImageFolder(root=ASSET_DIR, transform=get_augmentations())

In [169]:
targets = dataset.targets
indices = list(range(len(targets)))

train_idx, test_idx = train_test_split(
    indices,
    test_size=0.4,
    stratify=targets,
    random_state=int(time.time())
)

train_data = Subset(dataset, train_idx)
test_data = Subset(dataset, test_idx)

In [170]:
import numpy as np

train_labels = [dataset[i][1] for i in train_idx]
test_labels = [dataset[i][1] for i in test_idx]
print("Train label ratios:", np.bincount(train_labels) / len(train_labels))
print("Test label ratios:", np.bincount(test_labels) / len(test_labels))

Train label ratios: [0.04290429 0.03960396 0.04950495 0.04950495 0.04290429 0.04950495
 0.04950495 0.04950495 0.06270627 0.04290429 0.04950495 0.04290429
 0.04290429 0.04950495 0.04950495 0.04950495 0.03960396 0.04950495
 0.04290429 0.04950495 0.05610561]
Test label ratios: [0.04455446 0.03465347 0.04950495 0.04950495 0.04455446 0.04950495
 0.04950495 0.04950495 0.06435644 0.04455446 0.04950495 0.04455446
 0.04455446 0.04950495 0.04950495 0.04950495 0.03465347 0.04950495
 0.04455446 0.04950495 0.05445545]


In [171]:
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=len(test_labels))

In [172]:
def get_accuracy(pred: torch.Tensor, label: torch.Tensor) -> torch.Tensor:
    y_pred = torch.argmax(pred, dim=1).long()
    label = label.view(-1).long() 
    return (y_pred == label).float().mean()

def get_model_accuracy(model: nn.Module):
    with torch.no_grad():
        model.eval()

        sum_acc = 0
        cnt = 0

        for x, y in test_loader:
            pred = model(x)

            sum_acc += get_accuracy(pred, y)
            cnt += 1
        
        return float(sum_acc / cnt)

        

In [173]:
def train_model(loader: torch.utils.data.DataLoader, model: nn.Module):
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimiser = torch.optim.Adam(model.parameters(), lr=1e-3)
    loss_fn = nn.CrossEntropyLoss()  

    epoch_losses = []
    for i in range(100):
        epoch_loss = 0
        model.train()
        for _, (x, y) in enumerate(loader):
            optimiser.zero_grad()
            # x, y = x.cuda(), y.cuda()
            y_pred = model(x)
            
            loss = loss_fn(y_pred, y)
            epoch_loss += loss.item()
            loss.backward()
            optimiser.step()

        epoch_loss = epoch_loss / len(loader)
        epoch_losses.append(epoch_loss)
        print("Epoch: {}, Loss: {}, Accuracy: {}".format(i, epoch_loss, get_model_accuracy(model)))
        

    return model, epoch_losses

In [175]:
model = get_model(21)
train_model(train_loader, model)

Epoch: 0, Loss: 2.784492015838623, Accuracy: 0.1336633712053299
Epoch: 1, Loss: 1.9401527245839436, Accuracy: 0.20297029614448547
Epoch: 2, Loss: 1.4968406756718953, Accuracy: 0.3118811845779419
Epoch: 3, Loss: 1.1407219171524048, Accuracy: 0.4455445408821106
Epoch: 4, Loss: 0.8178398211797079, Accuracy: 0.47029703855514526
Epoch: 5, Loss: 0.6908033688863119, Accuracy: 0.4801980257034302
Epoch: 6, Loss: 0.5565394759178162, Accuracy: 0.5841584205627441
Epoch: 7, Loss: 0.4225753943125407, Accuracy: 0.603960394859314
Epoch: 8, Loss: 0.33182377616564435, Accuracy: 0.6435643434524536
Epoch: 9, Loss: 0.33034418026606244, Accuracy: 0.6782178282737732
Epoch: 10, Loss: 0.23564384877681732, Accuracy: 0.7178217768669128
Epoch: 11, Loss: 0.2177820255359014, Accuracy: 0.7722772359848022
Epoch: 12, Loss: 0.19691587487856546, Accuracy: 0.7722772359848022
Epoch: 13, Loss: 0.19119693338871002, Accuracy: 0.7772276997566223
Epoch: 14, Loss: 0.14457376301288605, Accuracy: 0.7277227640151978
Epoch: 15, Los

(Sequential(
   (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (1): LeakyReLU(negative_slope=0.1)
   (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (4): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (5): LeakyReLU(negative_slope=0.1)
   (6): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (8): Flatten(start_dim=1, end_dim=-1)
   (9): Linear(in_features=3136, out_features=70, bias=True)
   (10): LeakyReLU(negative_slope=0.1)
   (11): Dropout(p=0.3, inplace=False)
   (12): Linear(in_features=70, out_features=21, bias=True)
 ),
 [2.784492015838623,
  1.9401527245839436,
  1.4968406756718953,
  1.1407219171524048,
  0.8178398211797079,
  0.6908033688863119,
  0.5565394759178162,
  0.4225753943125407,
  0.33

In [176]:
torch.save(model.state_dict(), "image-classification-v0")

In [177]:
def get_accuracy_for_each_label(model: nn.Module):
    with torch.no_grad():
        model.eval()
        
        acc: List[int] = [0 for i in range(21)]
        cnt: List[int] = [0 for i in range(21)]

        for x, y in test_loader:
            for label in y:
                cnt[label] += 1
            
            pred = model(x)
            y_pred = torch.argmax(pred, dim=1).long()
            label = y.view(-1).long()

            for i in range(len(y_pred)):
                if y_pred[i] == label[i]:
                    acc[label[i]] += 1
                else:
                    print("WRONG LABEL, expected: {}, got: {}".format(label[i], y_pred[i]))

        for i in range(21):
            print("Total count: {}, label: {}, Accuracy: {}".format(cnt[i], i, float(acc[i] / cnt[i])))

get_accuracy_for_each_label(model)

WRONG LABEL, expected: 14, got: 13
WRONG LABEL, expected: 6, got: 2
WRONG LABEL, expected: 6, got: 2
Total count: 9, label: 0, Accuracy: 1.0
Total count: 7, label: 1, Accuracy: 1.0
Total count: 10, label: 2, Accuracy: 1.0
Total count: 10, label: 3, Accuracy: 1.0
Total count: 9, label: 4, Accuracy: 1.0
Total count: 10, label: 5, Accuracy: 1.0
Total count: 10, label: 6, Accuracy: 0.8
Total count: 10, label: 7, Accuracy: 1.0
Total count: 13, label: 8, Accuracy: 1.0
Total count: 9, label: 9, Accuracy: 1.0
Total count: 10, label: 10, Accuracy: 1.0
Total count: 9, label: 11, Accuracy: 1.0
Total count: 9, label: 12, Accuracy: 1.0
Total count: 10, label: 13, Accuracy: 1.0
Total count: 10, label: 14, Accuracy: 0.9
Total count: 10, label: 15, Accuracy: 1.0
Total count: 7, label: 16, Accuracy: 1.0
Total count: 10, label: 17, Accuracy: 1.0
Total count: 9, label: 18, Accuracy: 1.0
Total count: 10, label: 19, Accuracy: 1.0
Total count: 11, label: 20, Accuracy: 1.0


In [ ]:
# load v8 model
model_v8 = get_model(21)
model_v8.load_state_dict(torch.load("image-classification-v8", weights_only=True))

get_accuracy_for_each_label(model_v8)

torch.Size([202])
WRONG LABEL, expected: 17, got: 4
WRONG LABEL, expected: 20, got: 8
WRONG LABEL, expected: 20, got: 15
Total count: 9, label: 0, Accuracy: 1.0
Total count: 7, label: 1, Accuracy: 1.0
Total count: 10, label: 2, Accuracy: 1.0
Total count: 10, label: 3, Accuracy: 1.0
Total count: 9, label: 4, Accuracy: 1.0
Total count: 10, label: 5, Accuracy: 1.0
Total count: 10, label: 6, Accuracy: 1.0
Total count: 10, label: 7, Accuracy: 1.0
Total count: 13, label: 8, Accuracy: 1.0
Total count: 9, label: 9, Accuracy: 1.0
Total count: 10, label: 10, Accuracy: 1.0
Total count: 9, label: 11, Accuracy: 1.0
Total count: 9, label: 12, Accuracy: 1.0
Total count: 10, label: 13, Accuracy: 1.0
Total count: 10, label: 14, Accuracy: 1.0
Total count: 10, label: 15, Accuracy: 1.0
Total count: 7, label: 16, Accuracy: 1.0
Total count: 10, label: 17, Accuracy: 0.9
Total count: 9, label: 18, Accuracy: 1.0
Total count: 10, label: 19, Accuracy: 1.0
Total count: 11, label: 20, Accuracy: 0.8181818181818182


In [86]:
# Auto‑generate a loader snippet for the trained PyTorch model
from utils import generate_torch_loader_snippet

example_input = torch.tensor(
    [[0.0, 0.0]], dtype=torch.float32
)  # minimal example for tracing if needed
snippet = generate_torch_loader_snippet(
    model=get_model(21), prefer="auto"
, compression="zlib")

with open("image_model.py", "w") as f:
    f.write(snippet)